# State Management in LLM-Anwendungen

In diesem Notebook lernen wir, wie wir Zustände (State) in KI-Anwendungen verwalten können. Dies ist besonders wichtig für Konversationen und komplexe Workflows mit Large Language Models.

## 1. Warum ist State Management wichtig?

LLMs sind grundsätzlich zustandslos - sie haben keine inhärente Fähigkeit, sich an vorherige Interaktionen zu erinnern. Jede Anfrage wird isoliert betrachtet.

**Herausforderungen ohne State Management:**
- Keine Kontexterhaltung zwischen Anfragen
- Unmöglichkeit, auf vorherige Informationen zu referenzieren
- Keine Möglichkeit für mehrstufige Interaktionen

**Vorteile mit State Management:**
- Natürliche Konversationen durch Kontexterhaltung
- Effizienzsteigerung durch Vermeiden von Wiederholungen
- Möglichkeit für komplexe, mehrstufige Workflows

In [ ]:
# Benötigte Bibliotheken installieren
!pip install -q langchain langchain-openai langchain-community dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

from langchain_openai import ChatOpenAI

# LLM initialisieren
llm = ChatOpenAI(model="gpt-3.5-turbo")

## 2. Speichertypen in LangChain

LangChain bietet verschiedene Memory-Typen an, die für unterschiedliche Anwendungsfälle optimiert sind:

### 2.1 ConversationBufferMemory

Dies ist der einfachste Speichertyp - speichert den gesamten Konversationsverlauf als Liste von Nachrichten.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Erstellen des Speichers
buffer_memory = ConversationBufferMemory()

# Speichern von Kontext
buffer_memory.save_context({"input": "Hallo, ich bin Anna"}, {"output": "Hallo Anna! Wie kann ich dir helfen?"})
buffer_memory.save_context({"input": "Ich interessiere mich für maschinelles Lernen."},
                           {"output": "Das ist ein spannendes Thema! Möchtest du mehr darüber erfahren?"})

# Laden des gespeicherten Kontexts
print(buffer_memory.load_memory_variables({}))

In [ ]:
# Integration in eine Konversation
conversation = ConversationChain(
    llm=llm,
    memory=buffer_memory,
    verbose=True
)

response = conversation.predict(input="Wie heiße ich?")
print(response)

**Vorteile:**
- Einfach zu implementieren
- Vollständiger Konversationsverlauf verfügbar

**Nachteile:**
- Speicherbedarf wächst mit der Konversationslänge
- Probleme mit dem Kontextfenster des LLM bei langen Gesprächen

### 2.2 ConversationSummaryMemory

Dieses Memory fasst den Konversationsverlauf dynamisch zusammen, um den Speicherbedarf zu reduzieren.

In [ ]:
from langchain.memory import ConversationSummaryMemory

# Erstellen eines Summary Memory
summary_memory = ConversationSummaryMemory(llm=llm)

# Speichern von Kontext
summary_memory.save_context({"input": "Hallo, ich bin Michael und arbeite als Softwareentwickler."},
                            {"output": "Hallo Michael! Schön, einen Softwareentwickler kennenzulernen."})
summary_memory.save_context({"input": "Ich möchte eine KI-Anwendung für mein Unternehmen entwickeln."},
                            {"output": "Das klingt spannend! Welche Art von KI-Anwendung schwebt dir vor?"})
summary_memory.save_context({"input": "Eine, die Kundenfeedback automatisch analysieren kann."},
                            {"output": "Sentiment-Analyse ist ein guter Ansatz für die Analyse von Kundenfeedback."})

# Zusammenfassung anzeigen
print(summary_memory.load_memory_variables({}))

In [ ]:
# Integration in eine Konversation
summary_conversation = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)

response = summary_conversation.predict(input="Kannst du mir mehr über Sentiment-Analyse erzählen?")
print(response)

**Vorteile:**
- Effiziente Nutzung des Kontextfensters
- Gut für längere Konversationen

**Nachteile:**
- Kann Details verlieren
- Benötigt zusätzliche LLM-Aufrufe für die Zusammenfassung

### 2.3 VectorStoreMemory

Dieser Speichertyp nutzt Vektorähnlichkeiten, um relevante Teile früherer Konversationen abzurufen.

In [ ]:
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.memory import VectorStoreRetrieverMemory
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import faiss

# Embeddings erstellen
embeddings = OpenAIEmbeddings()
docstore = InMemoryDocstore({})

# Vector Store erstellen
vector_store = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(1536),  # Dimensionalität der OpenAI-Embeddings
    docstore=docstore,
    index_to_docstore_id={}
)

# Retriever erstellen
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# Vector Memory initialisieren
vector_memory = VectorStoreRetrieverMemory(retriever=retriever)

# Speichern von Kontext
vector_memory.save_context(
    {"input": "Mein Name ist Julia und ich bin Datenanalystin."},
    {"output": "Hallo Julia! Schön, dass du dich mit Datenanalyse beschäftigst."}
)
vector_memory.save_context(
    {"input": "Ich arbeite mit Python und nutze hauptsächlich Pandas und scikit-learn."},
    {"output": "Das sind hervorragende Tools für die Datenanalyse und maschinelles Lernen."}
)
vector_memory.save_context(
    {"input": "Ich möchte meine Fähigkeiten im Bereich Deep Learning verbessern."},
    {"output": "Für Deep Learning empfehle ich dir, TensorFlow oder PyTorch zu lernen."}
)

In [ ]:
# Abfrage des relevanten Kontexts
print(vector_memory.load_memory_variables({"input": "Mit welchen Tools arbeite ich?"})["history"])

**Vorteile:**
- Semantische Suche nach relevanten Informationen
- Nicht linear abhängig von der Konversationslänge

**Nachteile:**
- Komplexere Implementierung
- Benötigt Embedding-Modelle

### 2.4 Kombination verschiedener Memory-Typen

Für komplexere Anwendungen kann man verschiedene Memory-Typen kombinieren:

In [ ]:
from langchain.memory import CombinedMemory

# Zwei verschiedene Speichertypen erstellen
conv_memory = ConversationBufferMemory(memory_key="chat_history")
summary_memory_combined = ConversationSummaryMemory(llm=llm, memory_key="summary")

# Kombination der Speicher
combined_memory = CombinedMemory(memories=[conv_memory, summary_memory_combined])

# Speichern von Kontext
combined_memory.save_context(
    {"input": "Hallo, ich bin Thomas und interessiere mich für KI."},
    {"output": "Hallo Thomas! KI ist ein faszinierendes Thema."}
)
combined_memory.save_context(
    {"input": "Besonders interessiert mich der Bereich Natural Language Processing."},
    {"output": "NLP ist ein zentraler Bereich der KI mit vielen praktischen Anwendungen."}
)

# Gespeicherte Variablen anzeigen
print(combined_memory.load_memory_variables({}))

## 3. State Management in LangGraph

LangGraph erweitert die Möglichkeiten von LangChain mit zustandsbasierter Verarbeitung und bietet typisierte Zustände für komplexe Workflows.

In [ ]:
!pip install -q langgraph

In [ ]:
from typing import Annotated, TypedDict, List, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import operator
from langgraph.graph import StateGraph, END


# Definition eines typisierten Zustands
class ConversationState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]  # Liste von Nachrichten, die durch Operator '+' zusammengeführt werden
    next_step: str  # Kontrolle des Workflow-Flusses

In [ ]:
# Node-Funktionen definieren
def chat_node(state: ConversationState) -> ConversationState:
    """LLM-Node, der auf Benutzereingaben reagiert"""
    messages = state["messages"]
    response = llm.invoke(messages)

    # Überprüfen, ob eine Datenbank-Abfrage notwendig ist
    if "Datenbank" in messages[-1].content or "Suche" in messages[-1].content:
        return {"messages": messages + [response], "next_step": "database"}
    else:
        return {"messages": messages + [response], "next_step": "end"}


def database_query(state: ConversationState) -> ConversationState:
    """Simuliert eine Datenbankabfrage"""
    messages = state["messages"]

    # Einfache Simulation einer Datenbankabfrage
    system_message = AIMessage(content="Ich habe in der Datenbank folgende Informationen gefunden: ...")

    return {"messages": messages + [system_message], "next_step": "chat"}

In [ ]:
# Entscheidungsfunktion für den Workflow
def router(state: ConversationState) -> Literal["chat", "database", "end"]:
    return state["next_step"]

In [ ]:
# Graph erstellen
workflow = StateGraph(ConversationState)

# Knoten hinzufügen
workflow.add_node("chat", chat_node)
workflow.add_node("database", database_query)

# Startpunkt festlegen
workflow.set_entry_point("chat")

# Kanten mit Bedingungen hinzufügen
workflow.add_conditional_edges(
    "chat",
    router,
    {
        "database": "database",
        "end": END,
        "chat": "chat"
    }
)

# Verbindung von Datenbank zurück zum Chat
workflow.add_conditional_edges(
    "database",
    router,
    {
        "chat": "chat",
        "end": END,
        "database": "database"
    }
)

# Graph kompilieren
graph = workflow.compile()

In [ ]:
# Graph ausführen
result = graph.invoke({
    "messages": [HumanMessage(content="Hallo, ich suche Informationen zu maschinellem Lernen. Kannst du in der Datenbank nach Ressourcen suchen?")],
    "next_step": "chat"
})

# Ergebnis anzeigen
for message in result["messages"]:
    print(f"{message.type}: {message.content}\n")

## 4. Praxisübung: Chat-Anwendung mit Gedächtnis

Erstellen Sie eine einfache Chat-Anwendung, die sich an Benutzerpräferenzen erinnert und entsprechend reagiert.

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

# Prompt-Template mit Memory-Integration
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent, der sich an die Vorlieben und Informationen des Nutzers erinnert."),
    MessagesPlaceholder(variable_name="chat_history"),  # Platzhalter für den Chat-Verlauf
    ("human", "{input}")
])

# Erstellen der Konversationskette mit Buffer Memory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
chain = prompt | llm


# Funktion für die Chat-Interaktion
def chat(user_input):
    result = chain.invoke({
        "input": user_input,
        "chat_history": memory.load_memory_variables({}).get("chat_history", [])
    })

    # Speichern der Interaktion im Gedächtnis
    memory.save_context({"input": user_input}, {"output": result.content})

    return result.content

In [ ]:
# Beispiel-Interaktion
print("Assistant: " + chat("Hallo, ich bin Stefan und komme aus München."))
print("\nAssistant: " + chat("Ich mag Wandern und italienisches Essen."))
print("\nAssistant: " + chat("Kannst du mir eine Aktivität für das Wochenende empfehlen?"))
print("\nAssistant: " + chat("Wie heißt du nochmal und woher komme ich?"))

## 5. Übungsaufgaben

1. **Einfache Übung**: Modifizieren Sie die ConversationBufferMemory, um nur die letzten 3 Nachrichten zu speichern.

2. **Mittlere Übung**: Implementieren Sie eine Chat-Anwendung mit ConversationSummaryBufferMemory, die automatisch zusammenfasst, wenn der Kontext zu lang wird.

3. **Fortgeschrittene Übung**: Erweitern Sie den LangGraph-Workflow um einen zusätzlichen Knoten, der Benutzerpräferenzen in einer separaten Datenstruktur speichert und bei Bedarf abruft.

## 6. Zusammenfassung

- State Management ist entscheidend für die Entwicklung natürlicher und nützlicher KI-Anwendungen
- LangChain bietet verschiedene Memory-Typen für unterschiedliche Anwendungsfälle:
  - ConversationBufferMemory für einfache Konversationen
  - ConversationSummaryMemory für längere Gespräche
  - VectorStoreMemory für semantische Suche in der Konversationshistorie
- LangGraph erweitert die Möglichkeiten durch typisierte Zustände und komplexe Workflows
- Die Wahl des richtigen Memory-Typs hängt von den spezifischen Anforderungen der Anwendung ab

In der Praxis werden oft Kombinationen verschiedener Techniken verwendet, um optimale Ergebnisse zu erzielen.